In [1]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import plotly 
import plotly.plotly as py     
import plotly.graph_objs as go  
import cufflinks as cf
import pandas as pd
import folium

In [3]:
#data source: http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0111913
# Marcus Eriksen
# The dataset used in this model is based on expeditions from 2007–2013 (Table S1), surveying all five sub-tropical gyres (North Pacific, North Atlantic, South Pacific, South Atlantic, Indian Ocean) and extensive coastal regions and enclosed seas (Bay of Bengal, Australian coasts and the Mediterranean Sea)
# count density (pieces km−2) and weight density (g km−2)

eriksen_data = pd.read_csv('PlasticMarinePollutionGlobalDataset.csv')
eriksen_data.sample(3)

,Date,Latitude,Longitude,CD1 (/km^2),CD2 (/km^2),CD3 (/km^2),CD4 (/km^2),WD1 (g/km^2),WD2 (g/km^2),WD3 (g/km^2),WD4 (g/km^2),Sea State,Source,Info,Comments,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
1076,21/05/2008,43.3479,7.2387,NaN,NaN,NaN,1.96,NaN,NaN,NaN,176.40,2.50,F.Galgani,Macroplastic observations MED-2008-raw data-if...,NaN,NaN,NaN,NaN,NaN,NaN
1231,21/08/2008,43.4523,7.2010,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,1.50,F.Galgani,Macroplastic observations MED-2008-raw data-if...,NaN,NaN,NaN,NaN,NaN,NaN
134,31/01/2011,-34.8428,-42.5657,"3,303.52","8,809.38","1,651.76",NaN,0.91,34.38,106.81,NaN,2.00,M. Eriksen,SAG11-38,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#rename headers to clarify the size of the plastic
eriksen_data = eriksen_data.rename(columns={"CD1  (/km^2)": "Small Microplastics Count (/km^2)", 
                                            "CD2  (/km^2)": "Large Microplastics Count (/km^2)",
                                            "CD3  (/km^2)": "Mesoplastic Count (/km^2)", 
                                            "CD4  (/km^2)": "Macroplastic Count (/km^2)", 
                                            "WD1 (g/km^2)": "Small Microplastic Weight (g/km^2)", 
                                            "WD2 (g/km^2)": "Large Microplastics Weight (g/km^2)",
                                            "WD3 (g/km^2)": "Mesoplastic Weight (g/km^2)", 
                                            "WD4 (g/km^2)": "Macroplastic Weight (g/km^2)"})

In [44]:
#filter dataframe to only include North Pacific Gyre data
eriksen_data_NPG = eriksen_data.loc[eriksen_data["Info"].str.contains("NPG", na=False)]
eriksen_data_NPG.count()
#481 data pieces

Date                                   481
Latitude                               481
Longitude                              481
Small Microplastics Count              180
Large Microplastics Count (/km^2)      180
Mesoplastic Count (/km^2)              180
Macroplastic Count (/km^2)             301
Small Microplastic Weight (g/km^2)     180
Large Microplastics Weight (g/km^2)    180
Mesoplastic Weight (g/km^2)            180
Macroplastic Weight (g/km^2)           301
Sea State                              180
Source                                 481
Info                                   481
Comments                                10
Unnamed: 15                              0
Unnamed: 16                              0
Unnamed: 17                              0
Unnamed: 18                              0
Unnamed: 19                              0
dtype: int64

In [52]:
#not in date order. need to fix!!!
eriksen_data_NPG["Date"] = pd.to_datetime(eriksen_data_NPG["Date"])
eriksen_data_NPG.sort_values(by=["Date"], ascending=False)
eriksen_data_NPG

/Users/katiemunster/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Date,Latitude,Longitude,Small Microplastics Count,Large Microplastics Count (/km^2),Mesoplastic Count (/km^2),Macroplastic Count (/km^2),Small Microplastic Weight (g/km^2),Large Microplastics Weight (g/km^2),Mesoplastic Weight (g/km^2),Macroplastic Weight (g/km^2),Sea State,Source,Info,Comments,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
140,2007-09-16,37.2045,-133.8650,"15,506.45","18,552.36","1,107.60",NaN,2.90,36.88,61.06,NaN,4.00,C. Moore,NPG07-M001,NaN,NaN,NaN,NaN,NaN,NaN
141,2007-09-16,37.1755,-133.9495,"32,472.93","68,218.32","14,348.50",NaN,8.84,297.77,"1,158.28",NaN,4.00,C. Moore,NPG07-M002,NaN,NaN,NaN,NaN,NaN,NaN
142,2007-09-18,38.1515,-137.4531,"118,621.69","97,000.01","14,282.21",NaN,13.69,88.07,"3,654.46",NaN,3.00,C. Moore,NPG07-M004,NaN,NaN,NaN,NaN,NaN,NaN
143,2007-09-20,38.6913,-141.7720,"17,257.50","18,881.74","2,436.35",NaN,1.89,35.90,37.76,NaN,3.00,C. Moore,NPG07-M005,NaN,NaN,NaN,NaN,NaN,NaN
144,2007-09-20,38.6878,-142.0264,"5,100.93","14,353.77","1,660.77",NaN,1.87,22.04,224.67,NaN,3.00,C. Moore,NPG07-M006,NaN,NaN,NaN,NaN,NaN,NaN
145,2007-09-20,38.6410,-142.2665,"9,619.48","12,488.45",928.20,NaN,1.96,21.26,14.43,NaN,3.00,C. Moore,NPG07-M007,NaN,NaN,NaN,NaN,NaN,NaN
146,2007-09-21,37.8664,-143.8151,"16,235.81","255,134.23","11,597.01",NaN,3.71,500.30,361.83,NaN,4.00,C. Moore,NPG07-M008,NaN,NaN,NaN,NaN,NaN,NaN
147,2007-09-22,36.6822,-144.8449,"82,044.72","391,763.53","28,202.87",NaN,31.23,788.09,"12,882.05",NaN,4.00,C. Moore,NPG07-M010,NaN,NaN,NaN,NaN,NaN,NaN
148,2007-09-22,36.5084,-145.0356,"20,917.25","107,991.36","7,296.71",NaN,6.32,212.72,40.38,NaN,3.00,C. Moore,NPG07-M012,NaN,NaN,NaN,NaN,NaN,NaN
149,2007-09-24,32.5488,-146.9627,"22,362.54","60,773.50","7,498.03",NaN,6.04,454.99,775.32,NaN,4.00,C. Moore,NPG07-M013,NaN,NaN,NaN,NaN,NaN,NaN
